In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import pymongo
import json

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.astronaut_db
collection = db.facts

In [4]:
# URL of page to be scraped
url = 'https://www.archives.gov/research/alic/reference/space-timeline.html'

In [5]:
# Retrieve page with the requests module
response = requests.get(url)
print(response.url)

https://www.archives.gov/research/alic/reference/space-timeline.html


In [6]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text,'html.parser')

In [7]:
# Examine the results, then determine element that contains sought info
#print(soup.prettify())

In [8]:
# results are returned as an iterable list
results = soup.find_all('div', class_="region")
#results

In [9]:
#results = soup.find_all('div', class_="region")
#results

In [10]:
# Loop through returned results
years = soup.find_all('strong')
year_counts = []

for year in years:
    #print(year.text)
    
    year_counts.append(year.text) 

    
    year_list_length = (len(year_counts))
    
print(f'The length of the year list is {year_list_length} items')

The length of the year list is 47 items


In [11]:
year_counts.remove(year_counts[0])

In [12]:
numbered_years = []

for item in year_counts:
    year_floated = int(item)
    numbered_years.append(year_floated)
    
print(numbered_years)    

[1957, 1958, 1960, 1961, 1962, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1975, 1976, 1977, 1979, 1980, 1981, 1983, 1984, 1985, 1986, 1989, 1990, 1992, 1993, 1994, 1995, 1997, 1998, 1999, 2000, 2001, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]


In [13]:
#print(year_counts)
print(len(numbered_years))

46


In [14]:
### Loop through returned results for fun facts
fun_facts = soup.select('ul')
#print(fun_facts)
 
fact_list = []


for fact in fun_facts:
    first_li = fact('li')[0].text
    #print(fact)
    fact_list.append(first_li)
    #print('------------------')
    

    fact_list_length = (len(fact_list))
    
print(f'The length of the fact_list is {fact_list_length} items')
 

The length of the fact_list is 59 items


In [15]:
#print(fact_list)

In [16]:
updated_fact_list = fact_list[8:]
#updated_fact_list

In [17]:
usable_fact_list = updated_fact_list[:46]
#usable_fact_list

print(len(usable_fact_list))

46


In [18]:
df = pd.DataFrame({"Year": numbered_years, "Facts":usable_fact_list})
df

,Year,Facts
0,1957,October 4 - The Soviet Union launched the firs...
1,1958,January 31 - Explorer 1 was the first satellit...
2,1960,August 19 - The Soviet craft Sputnik 5 was lau...
3,1961,April 12 - Russian cosmonaut Yuri Gagarin beca...
4,1962,February 20 - Astronaut John Glenn became the ...
5,1965,"March 18 - While tethered to his spacecraft, c..."
6,1966,February 3 - The Russian spacecraft Luna 9 bec...
7,1967,"January 27 - Astronauts Gus Grissom, Ed White,..."
8,1968,September 15 - The Soviet spacecraft Zond 5 wa...
9,1969,"July 20 - Neil Armstrong and ""Buzz"" Aldrin bec..."


In [19]:
df.to_csv('year_fact.csv', index=False)

In [20]:
data = df.to_dict(orient='records')  # Here's our added param..

collection.insert_many(data)